In [1]:
#Make imports
import numpy as np
import re
import pickle
import os
import seaborn as sns
import string

In [2]:
!pip install englisttohindi

In [3]:
from englisttohindi.englisttohindi import EngtoHindi
trans = EngtoHindi(message="Hello  Everyone  ")
print(trans.convert)

सभी को नमस्कार


In [4]:
def preprocess(text):
  text = ''.join(ch for ch in text if ch not in string.punctuation)
  text = text.lower()
  text = re.sub(r'\d','',text)
  text = re.sub(r'\s+',' ',text)
  text = text.strip()
  return text

In [5]:
#Extract dataset and preprocess
dataset_root = "/content/"

if os.path.exists(dataset_root + "preprocessed_data.pickle"):
  with open(dataset_root + "preprocessed_data.pickle", 'rb') as f:
    english_sentences, hindi_sentences = pickle.load(f)
else:
  if not os.path.exists(dataset_root + "IITB.en-hi.en"):
    os.system("tar -xzf " + dataset_root + "parallel.tgz -C " + dataset_root)

  with open(dataset_root + "IITB.en-hi.en",'r') as f:
    english_sentences = f.read().split('\n')

  with open(dataset_root + "IITB.en-hi.hi",'r', encoding='utf-8') as f:
    hindi_sentences = f.read().split('\n')

  english_sentences = [preprocess(en) for en in english_sentences]
  hindi_sentences = ['<START> ' + re.sub('[a-zA-Z]','',preprocess(hi)) + ' <END>' for hi in hindi_sentences]

  #Remove duplicate sentences
  english_unique = set()
  english_sentences_temp = []
  hindi_sentences_temp = []
  #Use the minimum length to avoid IndexError
  l = min(len(english_sentences), len(hindi_sentences)) #Changed to use minimum length
  for i in range(l):
    if english_sentences[i] not in english_unique:
      english_unique.add(english_sentences[i])
      english_sentences_temp.append(english_sentences[i])
      hindi_sentences_temp.append(hindi_sentences[i])

  english_sentences = english_sentences_temp
  hindi_sentences = hindi_sentences_temp

  with open(dataset_root + "preprocessed_data.pickle",'wb') as f:
    pickle.dump((english_sentences, hindi_sentences), f)

In [6]:
len(english_sentences)

541472

In [ ]:
len(hindi_sentences)

36764

In [7]:
import json

with open('output_data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extract answers and translated_answers into separate lists
answers = data["answers"]
translated_answers = data["translated_answers"]

In [8]:
len(answers)

4082

In [9]:
len(translated_answers)

4082

In [10]:
english_sentences = english_sentences + answers
hindi_sentences = hindi_sentences + translated_answers

In [11]:
len(english_sentences)

545554

In [12]:
english_sentences[6011]

'bug buddy'

In [13]:
hindi_sentences[6011]

'<START> बग बड्डी <END>'

In [14]:
print(len(english_sentences), len(hindi_sentences))
print()
english_sentences[:3], hindi_sentences[:3]

545554 545554



(['give your application an accessibility workout',
  'accerciser accessibility explorer',
  'the default plugin layout for the bottom panel'],
 ['<START> अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें <END>',
  '<START> एक्सेर्साइसर पहुंचनीयता अन्वेषक <END>',
  '<START> निचले पटल के लिए डिफोल्ट प्लगइन खाका <END>'])

In [15]:
#Some parameters
vocab_size = 10000
total_sentences = 25000
maxlen = 10
epochs = 70
validation_split = 0.05

In [16]:
en_data = []
hi_data = []

cnt = 0

for (en,hi) in zip(english_sentences, hindi_sentences):
  l = min(len(en.split()), len(hi.split()))
  if l <= maxlen:
    en_data.append(en)
    hi_data.append(hi)
    cnt += 1
  if cnt == total_sentences:
    break

In [17]:
import tensorflow as tf

#Tokenize the texts and convert to sequences
en_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>', lower=False)
en_tokenizer.fit_on_texts(en_data)
en_sequences = en_tokenizer.texts_to_sequences(en_data)

hi_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>', lower=False)
hi_tokenizer.fit_on_texts(hi_data)
hi_sequences = hi_tokenizer.texts_to_sequences(hi_data)

english_vocab_size = len(en_tokenizer.word_index) + 1
hindi_vocab_size = len(hi_tokenizer.word_index) + 1
print("English Vocab Size: ", english_vocab_size)
print("Hindi Vocab Size: ", hindi_vocab_size)

English Vocab Size:  8020
Hindi Vocab Size:  9395


In [18]:
#Prepare encoder data
encoder_inputs = tf.keras.preprocessing.sequence.pad_sequences(en_sequences, maxlen=maxlen, padding='post')

In [19]:
#Prepare decoder data
decoder_inputs = []
decoder_outputs = []

for hi in hi_sequences:
  decoder_inputs.append(hi[:-1])
  decoder_outputs.append(hi[1:])

decoder_inputs = tf.keras.preprocessing.sequence.pad_sequences(decoder_inputs, maxlen=maxlen, padding='post')
decoder_outputs = tf.keras.preprocessing.sequence.pad_sequences(decoder_outputs, maxlen=maxlen, padding='post')

In [20]:
# Training and Testing split
# 95%, 5%
split = int(0.95 * total_sentences)

X_train = [encoder_inputs[:split], decoder_inputs[:split]]
y_train = decoder_outputs[:split]

# Test data to evaluate our NMT model using BLEU score
X_test = en_data[:split]
y_test = hi_data[:split]

print(X_train[0].shape, X_train[1].shape, y_train.shape)

(23750, 10) (23750, 10) (23750, 10)


In [22]:
#Define LSTM model
d_model = 256

#Encoder
inputs = tf.keras.layers.Input(shape=(None,))
x = tf.keras.layers.Embedding(english_vocab_size, d_model, mask_zero=True)(inputs)
_,state_h,state_c = tf.keras.layers.LSTM(d_model,activation='relu',return_state=True)(x)

#Decoder
targets = tf.keras.layers.Input(shape=(None,))
embedding_layer = tf.keras.layers.Embedding(hindi_vocab_size, d_model, mask_zero=True)
x = embedding_layer(targets)
decoder_lstm = tf.keras.layers.LSTM(d_model,activation='relu',return_sequences=True, return_state=True)
x,_,_ = decoder_lstm(x, initial_state=[state_h, state_c])
dense1 = tf.keras.layers.Dense(hindi_vocab_size, activation='softmax')
x = dense1(x)

model = tf.keras.models.Model(inputs=[inputs, targets],outputs=x)
model.summary()

loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, None, 256)      │      2,053,120 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_2 (NotEqual)    │ (None, None)           │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, None, 256)      │      2,405,120 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ [(None, 256), (None,   │        525,312 │ embedding_2[0][0],     │
│                           │ 256), (None, 256)]     │                │ not_equal_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ [(None, None, 256),    │        525,312 │ embedding_3[0][0],     │
│                           │ (None, 256), (None,    │                │ lstm_2[0][1],          │
│                           │ 256)]                  │                │ lstm_2[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, None, 9395)     │      2,414,515 │ lstm_3[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 7,923,379 (30.23 MB)

 Trainable params: 7,923,379 (30.23 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
#Save model after each epoch
save_model_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/en2hi.keras',
    monitor='val_accuracy',
    mode='max'
)

In [24]:
model.fit(X_train, y_train, epochs=epochs, validation_split=validation_split, callbacks=[save_model_callback, tf.keras.callbacks.TerminateOnNaN()])

Epoch 1/70
706/706 ━━━━━━━━━━━━━━━━━━━━ 29s 29ms/step - accuracy: 0.1012 - loss: 6.6504 - val_accuracy: 0.1068 - val_loss: 5.8181
Epoch 2/70
706/706 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.1239 - loss: 5.2551 - val_accuracy: 0.1151 - val_loss: 5.6502
Epoch 3/70
706/706 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.1446 - loss: 4.8547 - val_accuracy: 0.1236 - val_loss: 5.4861
Epoch 4/70
706/706 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.1596 - loss: 4.5557 - val_accuracy: 0.1263 - val_loss: 5.4402
Epoch 5/70
706/706 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.1735 - loss: 4.2885 - val_accuracy: 0.1323 - val_loss: 5.5474
Epoch 6/70
706/706 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.1889 - loss: 4.0352 - val_accuracy: 0.1349 - val_loss: 5.3658
Epoch 7/70
706/706 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.2032 - loss: 3.7955 - val_accuracy: 0.1375 - val_loss: 5.4573
Epoch 8/70
706/706 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.2146 - loss: 3.6123 - val_ac

In [25]:
#Retrieve previously saved stuff
saved_model = tf.keras.models.load_model('/content/en2hi.keras')
saved_model.summary()

inputs = saved_model.get_layer('input_layer').output
_,state_h,state_c = saved_model.get_layer('lstm').output
targets = saved_model.get_layer('input_layer_1').output
embedding_layer = saved_model.get_layer('embedding_1')
decoder_lstm = saved_model.get_layer('lstm_1')
dense1 = saved_model.get_layer('dense')

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, None, 256)      │      2,053,120 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_2 (NotEqual)    │ (None, None)           │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, None, 256)      │      2,405,120 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ [(None, 256), (None,   │        525,312 │ embedding_2[0][0],     │
│                           │ 256), (None, 256)]     │                │ not_equal_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ [(None, None, 256),    │        525,312 │ embedding_3[0][0],     │
│                           │ (None, 256), (None,    │                │ lstm_2[0][1],          │
│                           │ 256)]                  │                │ lstm_2[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, None, 9395)     │      2,414,515 │ lstm_3[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 15,846,760 (60.45 MB)

 Trainable params: 7,923,379 (30.23 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,923,381 (30.23 MB)

ValueError: No such layer: input_layer. Existing layers are: ['input_layer_2', 'input_layer_3', 'embedding_2', 'embedding_3', 'lstm_2', 'lstm_3', 'dense_1'].

In [ ]:
#Inference Model
#Encoder
encoder = tf.keras.models.Model(inputs, [state_h, state_c])

#Decoder
decoder_input_h = tf.keras.layers.Input(shape=(d_model,))
decoder_input_c = tf.keras.layers.Input(shape=(d_model,))
x = embedding_layer(targets)
x, decoder_output_h, decoder_output_c = decoder_lstm(x, initial_state=[decoder_input_h, decoder_input_c])
x = dense1(x)
decoder = tf.keras.models.Model([targets] + [decoder_input_h, decoder_input_c],[x] + [decoder_output_h, decoder_output_c])

In [ ]:
import numpy as np

def predict_sentence(en_input):
    # Convert input to numpy array
    input_seq = np.array(en_tokenizer.texts_to_sequences([en_input]))

    # Predict the initial hidden and cell states for the encoder
    next_h, next_c = encoder.predict(input_seq)

    # Prepare the first token as the <START> token with correct shape
    curr_token = np.zeros((1, 1))  # Reshape to (1, 1) for (batch_size, sequence_length)
    curr_token[0, 0] = hi_tokenizer.word_index['<START>']

    pred_sentence = ''

    for i in range(maxlen):
        # Predict the next word
        output, next_h, next_c = decoder.predict([curr_token, next_h, next_c])

        # Get the token with the highest probability
        next_token = np.argmax(output[0, 0, :])
        next_word = hi_tokenizer.index_word[next_token]

        # Check for the <END> token
        if next_word == '<END>':
            break
        else:
            pred_sentence += ' ' + next_word
            curr_token[0, 0] = next_token  # Update curr_token with correct shape

    return pred_sentence


In [ ]:
#Testing and Analysis
import nltk

candidates = []
references = []

ctr = 20
i = 0

while ctr>0:
  l = len(X_test[i].split())
  if l<=maxlen:   #Choose only sentences of length in range [5,15]
    pred_sentence = predict_sentence(X_test[i])
    candidates.append(pred_sentence.split())

    print("Input: ", X_test[i])
    print("Prediction: ", pred_sentence)

    trans = EngtoHindi(message=X_test[i])
    python_translated_sentence = trans.convert

    print("Google Translated Reference: ", python_translated_sentence)
    print("Dataset Reference: ", ' '.join(y_test[i].split()[1:-1]))
    print()
    references.append([y_test[i].split()[1:-1], python_translated_sentence.split()])

    ctr -= 1
  i += 1

print(nltk.translate.bleu_score.corpus_bleu(references, candidates))

In [ ]:
import numpy as np

def predict_sentence(en_input):
    # Step 1: Convert input text to sequence using the English tokenizer
    input_seq = np.array(en_tokenizer.texts_to_sequences([en_input]))

    # Step 2: Predict encoder hidden and cell states
    next_h, next_c = encoder.predict(input_seq)

    # Step 3: Initialize the first token as <START> for the decoder
    curr_token = np.zeros((1, 1))  # Shape: (batch_size, sequence_length)
    curr_token[0, 0] = hi_tokenizer.word_index.get('<START>', 1)  # Default to 1 if <START> not found

    pred_sentence = ''

    for _ in range(maxlen):
        # Step 4: Predict the next word
        output, next_h, next_c = decoder.predict([curr_token, next_h, next_c])

        # Step 5: Identify the word with the highest probability
        next_token = np.argmax(output[0, 0, :])
        next_word = hi_tokenizer.index_word.get(next_token, '')

        # Step 6: Check for <END> token to stop the translation
        if next_word == '<END>':
            break
        else:
            pred_sentence += ' ' + next_word
            curr_token[0, 0] = next_token  # Update current token for next prediction

    return pred_sentence.strip()


In [ ]:
user_input = input("Enter an English sentence to translate to Hindi: ")
preprocessed_input = preprocess(user_input)  # Preprocess input
translated_sentence = predict_sentence(preprocessed_input)
print("Translated Hindi sentence:", translated_sentence)

In [ ]:
#install transformers library
!pip install transformers -U -q

# install sentencepiece library
!pip install sentencepiece

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# download and save model
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
# import tokenizer
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")


In [ ]:
def translate_to_hindi(text):
    """
    Translates an English text input to Hindi.

    Args:
    text (str): The English text to be translated.

    Returns:
    str: The translated text in Hindi.
    """
    # Convert the input text to tensors
    model_inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True)

    # Translate from English to Hindi
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
    )

    # Decode the generated tokens to get the Hindi translation
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    return translation[0]

# Example usage
user_text = input("Enter a sentence in English to translate to Hindi: ")

trans = EngtoHindi(message=user_text)
print("Translation in Hindi:", translate_to_hindi(user_text))

python_translated_sentence = trans.convert
print("Python Translated Reference: ", python_translated_sentence)

Enter a sentence in English to translate to Hindi: The (the criminal act of deliberately setting fire to property.) led to extensive (the name for anything which can be owned.) damage .<n>The defendant 's legal team argued for an (the court's decision that a person is innocent of the crime they were charged with
Translation in Hindi: प्रतिवादी के कानूनी दल ने एक (न्यायालय के निर्णय के लिए तर्क दिया कि एक व्यक्ति उन अपराधों से निर्दोष है जिन पर वे आरोपित थे
Python Translated Reference:  (जानबूझकर संपत्ति में आग लगाने का आपराधिक कृत्य) से व्यापक क्षति हुई (किसी भी चीज़ का नाम जिसका स्वामित्व हो सकता है।) प्रतिवादी की कानूनी टीम ने अदालत के फैसले के लिए तर्क दिया कि एक व्यक्ति निर्दोष है 


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

def translate_to_hindi(text):
    """
    Translates an English text input to Hindi.

    Args:
    text (str): The English text to be translated.

    Returns:
    str: The translated text in Hindi.
    """
    # Convert the input text to tensors
    model_inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True)

    # Translate from English to Hindi
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
    )

    # Decode the generated tokens to get the Hindi translation
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    return translation[0]

# Example usage
user_text = input("Enter a sentence in English to translate to Hindi: ")
generated_translation = translate_to_hindi(user_text)

# Assuming `python_translated_sentence` is the reference translation from EngtoHindi library
trans = EngtoHindi(message=user_text)
python_translated_sentence = trans.convert  # Reference translation

# Tokenize the generated and reference translations
generated_tokens = word_tokenize(generated_translation)
reference_tokens = word_tokenize(python_translated_sentence)

# Calculate BLEU score
bleu_score = sentence_bleu([reference_tokens], generated_tokens)
print("Translation in Hindi:", generated_translation)
print("Python Translated Reference:", python_translated_sentence)
print("BLEU Score:", bleu_score)


Enter a sentence in English to translate to Hindi: the jury deliberated on charges of the criminal act of deliberately setting fire to property . led to extensive the name for anything which can be owned .'
Translation in Hindi: जौरी ने जानबूझकर संपत्ति पर आग लगाने के अपराध के आरोपों पर विचार-विमर्श किया. किसी भी संपत्ति के लिए विस्तृत नाम बनाने के लिए प्रेरित किया. '
Python Translated Reference: जूरी ने जानबूझकर संपत्ति में आग लगाने के आपराधिक कृत्य के आरोपों पर विचार-विमर्श किया। 
BLEU Score: 0.14887309680192354


In [ ]:
user_text = input("Enter a sentence in English to translate to Hindi: ")
generated_translation = translate_to_hindi(user_text)

# Assuming `python_translated_sentence` is the reference translation from EngtoHindi library
trans = EngtoHindi(message=user_text)
python_translated_sentence = trans.convert  # Reference translation

# Tokenize the generated and reference translations
generated_tokens = word_tokenize(generated_translation)
reference_tokens = word_tokenize(python_translated_sentence)

# Calculate BLEU score
bleu_score = sentence_bleu([reference_tokens], generated_tokens)
print("Translation in Hindi:", generated_translation)
print("Python Translated Reference:", python_translated_sentence)
print("BLEU Score:", bleu_score)

Enter a sentence in English to translate to Hindi: The (the criminal act of deliberately setting fire to property.) led to extensive (the name for anything which can be owned.) damage .<n>The defendant 's legal team argued for an (the court's decision that a person is innocent of the crime they were charged with).
Translation in Hindi: प्रतिवादी के कानूनी दल ने एक (न्यायालय के निर्णय के लिए तर्क दिया कि कोई व्यक्ति उन अपराधों से निर्दोष है जिन पर वह आरोपित किया गया था) के लिए तर्क दिया।
Python Translated Reference: (जानबूझकर संपत्ति में आग लगाने का आपराधिक कृत्य) से व्यापक क्षति हुई (किसी भी चीज़ का नाम जिसका स्वामित्व हो सकता है।) प्रतिवादी की कानूनी टीम ने अदालत के फैसले के लिए तर्क दिया कि एक व्यक्ति निर्दोष है 
BLEU Score: 0.11059204097669541
